<a href="https://colab.research.google.com/github/Lostenergydrink/anime-video-processing-colab/blob/main/Anime_Video_Processing_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anime Video Processing Notebook - Part 2: Frame Interpolation

This is Part 2 of the anime video processing notebook, focusing on frame interpolation to increase framerate from 24fps to higher rates like 60fps or 120fps.

**Note:** If you haven't run Part 1 yet and need the setup code, please run the Setup section below.

## 0. Setup (Only if you haven't run Part 1)

If you've already run Part 1, you can skip this section.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create directories
!mkdir -p /content/drive/MyDrive/anime_processing/input
!mkdir -p /content/drive/MyDrive/anime_processing/output/upscaled
!mkdir -p /content/drive/MyDrive/anime_processing/output/interpolated

# Install common dependencies
!apt-get update
!apt-get install -y ffmpeg
!pip install -q opencv-python matplotlib tqdm

In [ ]:
# Helper functions from Part 1
import os
import cv2
import time
import numpy as np
import subprocess
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def get_video_info(video_path):
    """Get information about a video file."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps if fps > 0 else 0
    
    cap.release()
    
    return {
        'width': width,
        'height': height,
        'fps': fps,
        'frame_count': frame_count,
        'duration': duration
    }

def display_video_info(video_path):
    """Display information about a video file in a nice format."""
    info = get_video_info(video_path)
    if info is None:
        print(f"Could not open video: {video_path}")
        return
    
    minutes, seconds = divmod(info['duration'], 60)
    hours, minutes = divmod(minutes, 60)
    
    html = f"""
    <div style="background-color: #f8f9fa; padding: 12px; border-radius: 6px; margin-bottom: 12px">
        <h3 style="margin-top: 0">Video Information</h3>
        <p><b>File:</b> {os.path.basename(video_path)}</p>
        <p><b>Resolution:</b> {info['width']}×{info['height']} pixels</p>
        <p><b>Frame Rate:</b> {info['fps']:.2f} fps</p>
        <p><b>Duration:</b> {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}</p>
        <p><b>Total Frames:</b> {info['frame_count']}</p>
    </div>
    """
    display(HTML(html))
    return info

def list_input_videos():
    """List all video files in the input directory."""
    input_dir = "/content/drive/MyDrive/anime_processing/input"
    video_extensions = [".mp4", ".mkv", ".avi", ".mov", ".webm"]
    videos = []
    
    for file in os.listdir(input_dir):
        if any(file.lower().endswith(ext) for ext in video_extensions):
            videos.append(os.path.join(input_dir, file))
    
    if not videos:
        print("No videos found in the input directory.")
        print(f"Please upload videos to: {input_dir}")
    else:
        print(f"Found {len(videos)} videos in the input directory:")
        for i, video in enumerate(videos, 1):
            print(f"{i}. {os.path.basename(video)}")
    
    return videos

def extract_frame(video_path, frame_number=0):
    """Extract a single frame from a video for preview."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_number >= frame_count:
        frame_number = frame_count // 2
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame
    else:
        return None

def show_frame(frame, title="Frame Preview"):
    """Display a video frame."""
    plt.figure(figsize=(12, 8))
    plt.title(title)
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

def format_time(seconds):
    """Format seconds into hours, minutes, seconds."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours}h {minutes}m {seconds}s"

## 1. Frame Interpolation Module

This module uses RIFE (Real-Time Intermediate Flow Estimation) to increase the frame rate of videos. This works well for anime content to create smoother motion.

### 1.1 Install RIFE

We'll install the RIFE algorithm for frame interpolation.

In [ ]:
# Clone the RIFE repository
!git clone https://github.com/hzwer/arXiv2020-RIFE
%cd arXiv2020-RIFE

# Install dependencies
!pip install -r requirements.txt
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

# Download pre-trained models
!mkdir -p train_log
!wget -nc https://github.com/hzwer/arXiv2020-RIFE/releases/download/v0.0/RIFE_trained_model_v4.6.zip
!unzip -n RIFE_trained_model_v4.6.zip -d train_log

### 1.2 List Available Videos

Let's see what videos are available in the input directory.

In [ ]:
# Return to the main directory
%cd /content
input_videos = list_input_videos()

### 1.3 Select a Video to Interpolate

In [ ]:
# Change this to the number of the video you want to interpolate
selected_video_index = 1  # e.g., 1 for the first video in the list

if input_videos and 1 <= selected_video_index <= len(input_videos):
    input_video_path = input_videos[selected_video_index - 1]
    print(f"Selected video: {os.path.basename(input_video_path)}")
    
    # Get and display video info
    video_info = display_video_info(input_video_path)
    
    # Show a preview frame
    preview_frame = extract_frame(input_video_path, frame_number=video_info['frame_count'] // 4)
    if preview_frame is not None:
        show_frame(preview_frame, "Input Video Preview")
else:
    print("Please upload videos to your Google Drive first or select a valid index.")

### 1.4 Interpolation Configuration

Set the parameters for interpolation.

In [ ]:
# Interpolation Settings
interpolation_settings = {
    'target_fps': 60,           # Target frame rate (e.g., 30, 60, 120)
    'scene_detection': True,    # Enable scene detection to prevent artifacts at cuts
    'scene_threshold': 15,      # Scene detection threshold (higher = less sensitive)
    'quality': 'high',          # Video quality: 'low', 'medium', 'high'
    'output_format': 'mp4',     # Output format: mp4, mkv
    'segment_size': 5,          # Minutes per segment (reduce if memory issues)
    'start_time': 0,            # Start time in seconds (0 = beginning)
    'duration': 0,              # Duration to process in seconds (0 = entire video)
    'skip_audio': False         # Skip audio processing (faster but no sound)
}

# Quality presets (CRF value, lower = better quality but larger file)
quality_presets = {
    'low': 28,
    'medium': 23,
    'high': 18
}

# Set output filename
if 'input_video_path' in locals():
    video_basename = os.path.splitext(os.path.basename(input_video_path))[0]
    output_filename = f"{video_basename}_{interpolation_settings['target_fps']}fps.{interpolation_settings['output_format']}"
    output_video_path = f"/content/drive/MyDrive/anime_processing/output/interpolated/{output_filename}"
    
    print(f"Output will be saved as: {output_filename}")
    
    # Estimate processing time
    if 'video_info' in locals():
        multiplier = interpolation_settings['target_fps'] / video_info['fps']
        frames_to_generate = video_info['frame_count'] * (multiplier - 1)
        estimated_time = 0.1 * frames_to_generate * (video_info['width'] * video_info['height'] / (1920 * 1080))
        print(f"Estimated processing time: {format_time(estimated_time)}")
        print("(This is a rough estimate. Actual time may vary based on video content and Colab's GPU availability)")

### 1.5 Interpolation Function

This function will handle video interpolation with progress tracking.

In [ ]:
def interpolate_video(input_path, output_path, settings):
    """Interpolate a video using RIFE."""
    # Verify input exists
    if not os.path.exists(input_path):
        print(f"Error: Input file {input_path} does not exist.")
        return False
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Get video info
    video_info = get_video_info(input_path)
    if video_info is None:
        print("Could not read video information.")
        return False
    
    # Calculate the appropriate exp value for RIFE based on target fps
    current_fps = video_info['fps']
    target_fps = settings['target_fps']
    
    # Check if target fps is achievable by doubling frames
    if target_fps / current_fps <= 1:
        print(f"Error: Target FPS ({target_fps}) must be higher than current FPS ({current_fps:.2f}).")
        return False
    
    # Determine exp value (number of times to double frames)
    multiplier = target_fps / current_fps
    exp = 1
    while 2**exp < multiplier:
        exp += 1
    
    # Calculate the actual output fps (may be slightly different from target)
    actual_fps = current_fps * (2**exp)
    
    print(f"Input FPS: {current_fps:.2f}")
    print(f"Target FPS: {target_fps}")
    print(f"Actual output FPS will be: {actual_fps:.2f} (using exp={exp})")
    
    # If not processing the entire video, extract segment
    working_input = input_path
    if settings['start_time'] > 0 or (settings['duration'] > 0 and settings['duration'] < video_info['duration']):
        working_input = '/content/segment_input.mp4'
        duration_param = f'-t {settings["duration"]}' if settings['duration'] > 0 else ''
        !ffmpeg -y -ss {settings['start_time']} {duration_param} -i "{input_path}" -c:v libx264 -crf 18 -preset fast "{working_input}"
        video_info = get_video_info(working_input)
    
    # If video is large, process in segments
    segment_size_frames = int(settings['segment_size'] * 60 * video_info['fps'])
    if video_info['frame_count'] > segment_size_frames and settings['segment_size'] > 0:
        segment_count = (video_info['frame_count'] + segment_size_frames - 1) // segment_size_frames
        print(f"Processing in {segment_count} segments...")
        
        # Create temporary directory for segments
        !mkdir -p /content/segments
        segments_output = []
        
        for i in range(segment_count):
            start_frame = i * segment_size_frames
            start_time = start_frame / video_info['fps']
            duration = min(settings['segment_size'] * 60, video_info['duration'] - start_time)
            
            print(f"\nProcessing segment {i+1}/{segment_count} (from {format_time(start_time)})")
            
            # Extract segment
            segment_input = f"/content/segments/segment_{i:03d}_input.mp4"
            !ffmpeg -y -ss {start_time} -t {duration} -i "{working_input}" -c:v libx264 -crf 18 -preset fast "{segment_input}"
            
            # Process segment
            segment_output = f"/content/segments/segment_{i:03d}_output.{settings['output_format']}"
            
            # Set up RIFE parameters
            scene_detection = '--skip' if not settings['scene_detection'] else ''
            audio_param = '--skip_audio' if settings['skip_audio'] else ''
            crf = quality_presets[settings['quality']]
            
            # Change to RIFE directory and run interpolation
            %cd /content/arXiv2020-RIFE
            !python inference_video.py \
                --exp={exp} \
                --video="{segment_input}" \
                --output="{segment_output}" \
                --scene_thresh={settings['scene_threshold']} \
                --crf={crf} \
                {scene_detection} \
                {audio_param}
            
            segments_output.append(segment_output)
        
        # Combine segments
        print("\nCombining segments...")
        segments_list = '/content/segments_list.txt'
        with open(segments_list, 'w') as f:
            for segment in segments_output:
                f.write(f"file '{segment}'\n")
        
        %cd /content
        !ffmpeg -y -f concat -safe 0 -i "{segments_list}" -c copy "{output_path}"
        
        # Clean up segments
        print("Cleaning up temporary files...")
        !rm -rf /content/segments
        !rm -f "{segments_list}"
    else:
        # Process entire video at once
        # Set up RIFE parameters
        scene_detection = '--skip' if not settings['scene_detection'] else ''
        audio_param = '--skip_audio' if settings['skip_audio'] else ''
        crf = quality_presets[settings['quality']]
        
        # Change to RIFE directory and run interpolation
        %cd /content/arXiv2020-RIFE
        !python inference_video.py \
            --exp={exp} \
            --video="{working_input}" \
            --output="{output_path}" \
            --scene_thresh={settings['scene_threshold']} \
            --crf={crf} \
            {scene_detection} \
            {audio_param}
    
    # Clean up temporary files
    %cd /content
    if working_input != input_path:
        !rm -f "{working_input}"
    
    # Verify output was created
    if os.path.exists(output_path):
        output_info = get_video_info(output_path)
        if output_info:
            print(f"\nInterpolation complete! Output saved to: {output_path}")
            print(f"Original FPS: {video_info['fps']:.2f}")
            print(f"New FPS: {output_info['fps']:.2f}")
            return True
    
    print("Error: Failed to create output file.")
    return False

### 1.6 Run Interpolation

Now let's interpolate the selected video.

In [ ]:
# Only run if we have selected a valid video
if 'input_video_path' in locals() and 'output_video_path' in locals():
    print("Starting interpolation process...")
    start_time = time.time()
    
    success = interpolate_video(input_video_path, output_video_path, interpolation_settings)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    if success:
        print(f"Total processing time: {format_time(elapsed_time)}")
        
        # Compare before and after
        input_frame = extract_frame(input_video_path, frame_number=get_video_info(input_video_path)['frame_count'] // 4)
        output_frame = extract_frame(output_video_path, frame_number=get_video_info(output_video_path)['frame_count'] // 4)
        
        if input_frame is not None and output_frame is not None:
            plt.figure(figsize=(18, 8))
            
            plt.subplot(1, 2, 1)
            plt.title(f"Original ({video_info['fps']:.2f} fps)")
            plt.imshow(input_frame)
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.title(f"Interpolated ({get_video_info(output_video_path)['fps']:.2f} fps)")
            plt.imshow(output_frame)
            plt.axis('off')
            
            plt.tight_layout()
            plt.show()
    else:
        print("Interpolation failed.")
else:
    print("Please select a valid input video first.")

## 2. Audio Synchronization (Optional)

If you encounter audio sync issues after interpolation, this section can help fix them.

In [ ]:
def fix_audio_sync(video_path, delay_seconds, output_path=None):
    """Fix audio sync by adding a delay to the audio track."""
    if not os.path.exists(video_path):
        print(f"Error: Video file {video_path} does not exist.")
        return False
    
    if output_path is None:
        base, ext = os.path.splitext(video_path)
        output_path = f"{base}_synced{ext}"
    
    # Extract audio
    audio_path = "/content/temp_audio.aac"
    !ffmpeg -y -i "{video_path}" -vn -acodec copy "{audio_path}"
    
    # Add delay to audio and combine with video
    !ffmpeg -y -i "{video_path}" -itsoffset {delay_seconds} -i "{audio_path}" \
        -map 0:v -map 1:a -c:v copy -c:a aac -shortest "{output_path}"
    
    # Clean up
    !rm -f "{audio_path}"
    
    if os.path.exists(output_path):
        print(f"Audio sync fixed. Output saved to: {output_path}")
        return True
    else:
        print("Error: Failed to create output file.")
        return False

In [ ]:
# Use this cell to fix audio sync issues
# Example:
# video_with_sync_issue = "/content/drive/MyDrive/anime_processing/output/interpolated/your_video.mp4"
# fix_audio_sync(video_with_sync_issue, delay_seconds=5)  # Add 5 second delay to audio

## 3. Frame Comparison Visualization

This section helps visualize the difference between original and interpolated frames.

In [ ]:
def compare_frames(original_video, interpolated_video, frame_index=None):
    """Compare frames from original and interpolated videos."""
    original_info = get_video_info(original_video)
    interpolated_info = get_video_info(interpolated_video)
    
    if original_info is None or interpolated_info is None:
        print("Could not read video information.")
        return
    
    # Calculate frame index if not provided
    if frame_index is None:
        frame_index = original_info['frame_count'] // 4
    
    # Calculate corresponding frame in interpolated video
    interpolated_frame_index = int(frame_index * (interpolated_info['fps'] / original_info['fps']))
    
    # Extract frames
    original_frame = extract_frame(original_video, frame_number=frame_index)
    interpolated_frame = extract_frame(interpolated_video, frame_number=interpolated_frame_index)
    
    if original_frame is None or interpolated_frame is None:
        print("Could not extract frames.")
        return
    
    # Display frames side by side
    plt.figure(figsize=(18, 8))
    
    plt.subplot(1, 2, 1)
    plt.title(f"Original ({original_info['fps']:.2f} fps)")
    plt.imshow(original_frame)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.title(f"Interpolated ({interpolated_info['fps']:.2f} fps)")
    plt.imshow(interpolated_frame)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Use this to compare frames from the original and interpolated videos
# Example:
# if 'input_video_path' in locals() and 'output_video_path' in locals() and os.path.exists(output_video_path):
#     compare_frames(input_video_path, output_video_path)

## 4. Tips for Best Results

1. **Scene Detection**: For anime, set scene_threshold between 15-20 to properly detect cuts
2. **Segment Size**: If you encounter memory issues, reduce the segment_size parameter
3. **Target FPS**: For best results with anime, use 60fps rather than 120fps
4. **Audio Issues**: If audio gets out of sync, use the fix_audio_sync function
5. **Quality**: Use 'high' quality for final renders, 'medium' or 'low' for tests
6. **Colab Limitations**: Remember that Colab sessions have time limits; save your work regularly